In [6]:
from turtle import width
import torch
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [7]:
class ImagePixelDataset(torch.utils.data.Dataset):

  def __init__(self,image_path):
    self.image = np.array(Image.open(image_path))
    self.height,self.width,_ = self.image.shape

  def __len__(self):
    return self.height * self.width

  def __getitem__(self, idx):

    y = idx // self.width
    x = idx % self.width
    color = self.image[y,x]/255

    y/= self.height
    x/= self.width

    return np.array([x,y]), color

In [8]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

path = "/content/drive/My Drive/sifnos-greece-3840x2160-12799.jpg"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
dataset = ImagePixelDataset(path)

dataloader = DataLoader(dataset,batch_size=16,shuffle=True)
train_feature, train_labels = next(iter(dataloader))

In [10]:
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return x

class MLP3(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2, output_size):
        super(MLP3, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        return x

class MLP4(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3, output_size):
        super(MLP4, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, output_size)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        return x

In [11]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
input_size = 2 # Define your input size based on the number of pixels in an image
hidden_size = 256  # You can adjust this based on your requirements
output_size = 3  # Assuming RGB prediction

In [ ]:
model_256 = MLP(input_size, hidden_size, output_size)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model_256.parameters(), lr=0.01)

Training 2 layer

In [ ]:
from tqdm import tqdm

num_epochs = 3
for epoch in range(num_epochs):
    for inputs, targets in tqdm(train_dataloader):
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        optimizer.zero_grad()

        # Forward pass
        outputs = model_256(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')


  0%|          | 0/414720 [00:00<?, ?it/s]<ipython-input-24-fe6654b23c18>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-24-fe6654b23c18>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)
100%|██████████| 414720/414720 [07:55<00:00, 871.52it/s]


Epoch 1/3, Loss: 0.0599


100%|██████████| 414720/414720 [07:55<00:00, 872.48it/s] 


Epoch 2/3, Loss: 0.1038


100%|██████████| 414720/414720 [07:43<00:00, 894.31it/s]

Epoch 3/3, Loss: 0.0432


In [ ]:
model_256.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_256(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-25-0da2bb3ac985>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-25-0da2bb3ac985>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.1075


In [ ]:
torch.save(model_256, '/content/drive/My Drive/model_mlp_2_256_3.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

<ipython-input-16-bdd342f0fc17>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coor = torch.tensor(coor)


In [ ]:
batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_256(batch_coor)
    pre.append(batch_pred)


<ipython-input-17-b9e9c0f3fc68>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)


In [ ]:
predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_2_256_3.png')

Training 3 layer

In [ ]:
input_size = 2 # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction


model_256_128 = MLP3(input_size,256,128, output_size)


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_256_128.parameters(), lr=0.01)


num_epochs = 2
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        optimizer.zero_grad()

        # Forward pass
        outputs = model_256_128(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

<ipython-input-10-64d3c340710c>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-10-64d3c340710c>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Epoch 1/2, Loss: 0.0357
Epoch 2/2, Loss: 0.0202


In [ ]:
model_256_128.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_256_128(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-11-25bffee3307f>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-11-25bffee3307f>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0380


In [ ]:
torch.save(model_256_128, '/content/drive/My Drive/model_mlp_2_256_128_3.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_256_128(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_2_256_128_3.png')

<ipython-input-13-39939a512046>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coor = torch.tensor(coor)
<ipython-input-13-39939a512046>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)


Training 4 layer

In [ ]:
input_size = 2 # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction


model_256_128_64 = MLP4(input_size,256,128,64, output_size)


criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model_256_128_64.parameters(), lr=0.01)


num_epochs = 2
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        optimizer.zero_grad()

        # Forward pass
        outputs = model_256_128_64(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

<ipython-input-35-5f213a06931f>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-35-5f213a06931f>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Epoch 1/2, Loss: 0.0385
Epoch 2/2, Loss: 0.0713


In [ ]:
model_256_128_64.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_256_128_64(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-36-acf6ac663735>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-36-acf6ac663735>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0724


In [ ]:
torch.save(model_256_128_64, '/content/drive/My Drive/model_mlp_2_256_128_64_3.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_256_128_64(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_2_256_128_64_3.png')

<ipython-input-38-4a7a10a82288>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)


In [ ]:
batch_size = 1024
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_size = 2 # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction


model_256_128_64_b1024 = MLP4(input_size,256,128,64, output_size)


criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model_256_128_64_b1024.parameters(), lr=0.01)


num_epochs = 2
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        optimizer.zero_grad()

        # Forward pass
        outputs = model_256_128_64_b1024(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

<ipython-input-39-1a7634670891>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-39-1a7634670891>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Epoch 1/2, Loss: 0.0937
Epoch 2/2, Loss: 0.0900


In [ ]:
model_256_128_64_b1024.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_256_128_64_b1024(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-40-fe8a7c6bc17e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-40-fe8a7c6bc17e>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0921


In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_256_128_64_b1024(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_2_256_128_64_b1024.png')

<ipython-input-41-0fad2ac12d73>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)


Higher Dimention

In [1]:
from turtle import width
import torch
from PIL import Image
from torch.utils.data import DataLoader
import numpy as np

In [12]:
class ImagePixelDatasetHigherDimention(torch.utils.data.Dataset):

  def __init__(self,image_path,L):
    self.image = np.array(Image.open(image_path))
    self.height,self.width,_ = self.image.shape
    self.L = L


  def __len__(self):
    return self.height * self.width

  def __getitem__(self, idx):

    y = idx // self.width
    x = idx % self.width
    color = self.image[y,x]/255

    y = np.interp(y,[0,self.height],[-1,1])
    x = np.interp(x,[0,self.width],[-1,1])

    input = []

    for l in range(0,self.L):
        input.append(np.sin(2**l * np.pi * x))
        input.append(np.cos(2**l * np.pi * x))

    for l in range(0,self.L):
        input.append(np.sin(2**l * np.pi * y))
        input.append(np.cos(2**l * np.pi * y))


    return np.array(input), color

In [13]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

path = "/content/drive/My Drive/sifnos-greece-3840x2160-12799.jpg"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
L=4

dataset = ImagePixelDatasetHigherDimention(path,L)
dataloader = DataLoader(dataset,batch_size=16,shuffle=True)
train_feature, train_labels = next(iter(dataloader))
print(train_feature, train_labels)

tensor([[-0.8983,  0.4394, -0.7894, -0.6139,  0.9692, -0.2462, -0.4772, -0.8788,
         -0.7274, -0.6862,  0.9983, -0.0581, -0.1161, -0.9932,  0.2306,  0.9730],
        [-0.5474, -0.8369,  0.9162,  0.4007,  0.7343, -0.6788, -0.9969, -0.0785,
          0.5075,  0.8616,  0.8746,  0.4848,  0.8480, -0.5299, -0.8988, -0.4384],
        [ 0.2683, -0.9633, -0.5169,  0.8560, -0.8850,  0.4656, -0.8241, -0.5664,
          0.6884,  0.7254,  0.9986,  0.0523,  0.1045, -0.9945, -0.2079,  0.9781],
        [-0.5127,  0.8586, -0.8804,  0.4743, -0.8351, -0.5501,  0.9188, -0.3947,
          0.6691, -0.7431, -0.9945,  0.1045, -0.2079, -0.9781,  0.4067,  0.9135],
        [ 1.0000,  0.0016,  0.0033, -1.0000, -0.0065,  1.0000, -0.0131,  0.9999,
          0.6065,  0.7951,  0.9644,  0.2644,  0.5100, -0.8601, -0.8774,  0.4797],
        [ 0.2683,  0.9633,  0.5169,  0.8560,  0.8850,  0.4656,  0.8241, -0.5664,
          0.5025,  0.8646,  0.8689,  0.4950,  0.8601, -0.5100, -0.8774, -0.4797],
        [ 0.0850, -0.9

In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, output_size)
        self.relu4 = nn.ReLU()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        return x

In [ ]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
input_size = 4*L # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction


model_hd = MLP(input_size,256,128,64, output_size)


criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model_hd.parameters(), lr=0.01)


num_epochs = 3
for epoch in range(num_epochs):
    for inputs, targets in train_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        optimizer.zero_grad()

        # Forward pass
        outputs = model_hd(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

<ipython-input-49-8b7b4e4a07b6>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-49-8b7b4e4a07b6>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Epoch 1/3, Loss: 0.0212
Epoch 2/3, Loss: 0.0161
Epoch 3/3, Loss: 0.0205


In [ ]:
model_hd.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_hd(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-50-76934a52435c>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-50-76934a52435c>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0236


In [ ]:

torch.save(model_hd, '/content/drive/My Drive/model_mlp_hd.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

model_hd.eval()

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_hd(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_hd.png')

<ipython-input-52-2253b964bab2>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)


In [ ]:
L=2

dataset = ImagePixelDatasetHigherDimention(path,L)
dataloader = DataLoader(dataset,batch_size=16,shuffle=True)
train_feature, train_labels = next(iter(dataloader))
print(train_feature, train_labels)

tensor([[ 0.0376,  0.9993,  0.0752,  0.9972,  0.9436, -0.3311, -0.6248, -0.7808],
        [ 1.0000,  0.0065,  0.0131, -0.9999,  0.9929, -0.1190, -0.2363, -0.9717],
        [-0.3675, -0.9300,  0.6836,  0.7299,  0.9492,  0.3145,  0.5972, -0.8021],
        [-0.3507,  0.9365, -0.6569,  0.7540,  0.9387,  0.3448,  0.6472, -0.7623],
        [-0.3323,  0.9432, -0.6268,  0.7792, -0.0262,  0.9997, -0.0523,  0.9986],
        [-0.9978,  0.0670, -0.1338, -0.9910,  0.9969, -0.0785, -0.1564, -0.9877],
        [ 0.9393,  0.3430,  0.6445, -0.7646, -0.4436, -0.8962,  0.7951,  0.6065],
        [-0.9882,  0.1532, -0.3028, -0.9531, -0.9962,  0.0872, -0.1736, -0.9848],
        [-0.9788,  0.2047, -0.4007, -0.9162,  0.9799, -0.1994, -0.3907, -0.9205],
        [-0.4829, -0.8757,  0.8457,  0.5336, -0.8802, -0.4746,  0.8355, -0.5495],
        [-0.9382, -0.3461,  0.6494, -0.7604, -0.8004, -0.5995,  0.9596, -0.2812],
        [ 0.9960, -0.0899, -0.1790, -0.9838, -0.0756, -0.9971,  0.1507,  0.9886],
        [ 0.8997

In [ ]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from tqdm import tqdm

input_size = 4*L # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction


model_hd_l2 = MLP4(input_size,256,128,64, output_size)


criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model_hd_l2.parameters(), lr=0.01)


num_epochs = 3
for epoch in range(num_epochs):
    for inputs, targets in tqdm(train_dataloader):
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        optimizer.zero_grad()

        # Forward pass
        outputs = model_hd_l2(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

  0%|          | 0/414720 [00:00<?, ?it/s]<ipython-input-19-9ae4fbe6dfb1>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-19-9ae4fbe6dfb1>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)
100%|██████████| 414720/414720 [14:21<00:00, 481.28it/s]


Epoch 1/3, Loss: 0.0299


100%|██████████| 414720/414720 [14:15<00:00, 484.63it/s]


Epoch 2/3, Loss: 0.0245


100%|██████████| 414720/414720 [14:03<00:00, 491.88it/s]

Epoch 3/3, Loss: 0.0076


In [ ]:
model_hd_l2.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_hd_l2(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-20-00bcd8139e86>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-20-00bcd8139e86>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0269


In [ ]:
torch.save(model_hd_l2, '/content/drive/My Drive/model_mlp_hd_l2.pt')

In [ ]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

model_hd_l2.eval()

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_hd_l2(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_hd_l2.png')

In [14]:
L=1

dataset = ImagePixelDatasetHigherDimention(path,L)
dataloader = DataLoader(dataset,batch_size=16,shuffle=True)
train_feature, train_labels = next(iter(dataloader))
print(train_feature, train_labels)

tensor([[-0.4394, -0.8983,  0.3475, -0.9377],
        [ 0.5364, -0.8440,  0.7490, -0.6626],
        [-0.9989,  0.0474,  0.7451,  0.6670],
        [-0.5141,  0.8577, -0.9853, -0.1708],
        [ 0.1257,  0.9921,  0.3090, -0.9511],
        [ 0.4438, -0.8961,  0.1103,  0.9939],
        [ 0.7914, -0.6114,  0.4226,  0.9063],
        [-0.9847, -0.1742, -0.6926, -0.7214],
        [-0.5253,  0.8509, -0.0029,  1.0000],
        [ 0.9967,  0.0817,  0.8802, -0.4746],
        [-0.1516, -0.9884,  0.7771, -0.6293],
        [-0.2271,  0.9739, -0.4173,  0.9088],
        [-0.3584,  0.9336, -0.9724, -0.2334],
        [ 0.4685, -0.8835,  0.0436,  0.9990],
        [ 0.9521,  0.3059,  0.2193,  0.9757],
        [-0.2919,  0.9565,  0.8208,  0.5712]], dtype=torch.float64) tensor([[0.2510, 0.2824, 0.1922],
        [0.0314, 0.0627, 0.0196],
        [0.3255, 0.4157, 0.3529],
        [0.2039, 0.1608, 0.1451],
        [0.1137, 0.1608, 0.2627],
        [0.0078, 0.2157, 0.5137],
        [0.0118, 0.2000, 0.4431],
    

In [15]:
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])



In [16]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
from tqdm import tqdm

input_size = 4*L # Define your input size based on the number of pixels in an image
output_size = 3  # Assuming RGB prediction


model_hd_l1 = MLP4(input_size,256,128,64, output_size)


criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model_hd_l1.parameters(), lr=0.01)


num_epochs = 3
for epoch in range(num_epochs):
    for inputs, targets in tqdm(train_dataloader):
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        optimizer.zero_grad()

        # Forward pass
        outputs = model_hd_l1(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

  0%|          | 0/414720 [00:00<?, ?it/s]<ipython-input-18-e7b514f68dda>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-18-e7b514f68dda>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)
100%|██████████| 414720/414720 [14:06<00:00, 489.99it/s]


Epoch 1/3, Loss: 0.0292


100%|██████████| 414720/414720 [13:51<00:00, 498.48it/s]


Epoch 2/3, Loss: 0.0256


100%|██████████| 414720/414720 [13:39<00:00, 506.18it/s]

Epoch 3/3, Loss: 0.0148


In [19]:
model_hd_l1.eval()
with torch.no_grad():
    test_loss = 0.0
    for inputs, targets in test_dataloader:
        # Flatten the inputs if needed

        inputs = torch.tensor(inputs,dtype=torch.float32)
        targets = torch.tensor(targets,dtype=torch.float32)

        outputs = model_hd_l1(inputs)

        test_loss += criterion(outputs, targets)

    average_test_loss = test_loss / len(test_dataloader)
    print(f'Test Loss: {average_test_loss:.4f}')

<ipython-input-19-01dd8aa317fa>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs,dtype=torch.float32)
<ipython-input-19-01dd8aa317fa>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets,dtype=torch.float32)


Test Loss: 0.0292


In [20]:
coor = [dataset[i][0] for i in range(len(dataset))]
coor = torch.tensor(coor)

model_hd_l1.eval()

batch_size = 64
with torch.no_grad():
  pre = []
  for i in range(0, coor.size(0), batch_size):
    batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
    batch_pred = model_hd_l1(batch_coor)
    pre.append(batch_pred)

predicted_rgb = torch.cat(pre, dim=0)

predicted_rgb = predicted_rgb.view(dataset.height, dataset.width, 3).numpy() * 255
predicted_image = Image.fromarray(predicted_rgb.astype('uint8'))

predicted_image.save('./mlp_hd_l1.png')

<ipython-input-20-834b05bb3df6>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coor = torch.tensor(coor)
<ipython-input-20-834b05bb3df6>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_coor = torch.tensor(coor[i:i+batch_size, :],dtype=torch.float32)
